In [18]:
import os
import sys 
!pip install --user numpy
import numpy as nd 
print(os.getcwd())
#!pip install --user nba_api
#!pip install --user pandas
#!pip install --user requests
print(sys.version)
print(sys.executable)
#!{sys.executable} -m pip install --user nba_api


C:\Users\Temesgen\REPOS\NBAStats\addFullLeaugeSummary1Year\NBASTATS
3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)]
C:\Users\Temesgen\anaconda3\python.exe


In [10]:
headers  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [12]:


import nba_api
import pandas as pd
import requests
import time
from IPython.display import display, HTML
from nba_api.stats.static import teams

from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import playbyplayv2

pd.options.display.max_columns = None
pd.options.display.max_rows = None


In [13]:
#Get information about playyers

#nba_players = players.get_players()
#df = pd.DataFrame(nba_players)
#df.sample(5)



active_players = players.get_active_players()
df = pd.DataFrame(active_players)
df.sample(10)

active_players = players.get_active_players()
df = pd.DataFrame(active_players)

booker = df.loc[(df["last_name"] == "Booker")]
booker

,id,full_name,first_name,last_name,is_active
50,1626164,Devin Booker,Devin,Booker,True


In [14]:
#Get Team Information
nba_teams = teams.get_teams()
allteams = pd.DataFrame(nba_teams)


suns = teams.find_team_by_abbreviation('phx')
#suns


# get game logs from the reg season
gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2022-23',league_id_nullable='00',team_id_nullable="1610612756",season_type_nullable='Regular Season')
#print(type(gamefinder))
games = gamefinder.get_data_frames()[0]
game_ids = games["GAME_ID"].to_list()

f = open("demofile3.txt", "w")

#for game in game_ids:
#    pbp = playbyplayv2.PlayByPlayV2(game)
#    pbp = pbp.get_data_frames()[0]
#    f.write(pbp.to_string())
#    time.sleep(1)
f.close()


In [16]:
def logRuns(pbp, team):
    f = open("demofile3.txt", "w")
    g = open("demofile4.txt", "w")
    timeout = (team.upper() + " Timeout")
    
    g.write(pbp.to_string())
    g.close()
  
    
    btat = []
    #print(team.upper() + " Timeout" + pbp.iloc[47]['VISITORDESCRIPTION'])
    
    pbpf = pbp.loc[(pbp['SCORE'].notna()) |  (pbp['HOMEDESCRIPTION'].str.contains(team.upper() + " Timeout"))  |  (pbp['VISITORDESCRIPTION'].str.contains(team.upper() + " Timeout")) 
                  |  (pbp['HOMEDESCRIPTION'].str.contains(team + " Timeout"))  |  (pbp['VISITORDESCRIPTION'].str.contains(team + " Timeout"))]
    pbpf = pbpf.reset_index(drop=True)
    f.write(pbpf.to_string())
    f.close()
    
    
    for index,play in pbpf.iterrows():
#            print(str(index))
                                              
        playHD = str(play["HOMEDESCRIPTION"]).upper()
        vistHD = str(play["VISITORDESCRIPTION"]).upper()
        if timeout.upper() in playHD:

            #was there a run?
            
            if index > 10 and index + 10 < len(pbpf) :
               # print(str(index))
                for i in range(0,5):
                    if  pbpf.iloc[index -(11 + i)]['SCORE'] != None:
                        startscore = pbpf.iloc[index -(11 + i)]['SCORE'].split(" - ")
              
                if  pbpf.iloc[index -1]['SCORE'] == None:
                    break
                
                endscore = pbpf.iloc[index -1]['SCORE'].split(" - ")
                
                homediff = int(endscore[1])-int(startscore[1])
                visitdiff = int(endscore[0])-int(startscore[0])
                bt_diff = visitdiff -homediff
                
                #print(str(visitdiff -homediff))
                #print(startscore)
                #print(endscore)
                
                if index + 15 < len(pbpf) and bt_diff >= 7:

                    startscore = pbpf.iloc[index - 1]['SCORE'].split(" - ")
                    
                    for i in range(0,5):
                        if  pbpf.iloc[index + (10 + i)]['SCORE'] != None:
                            endscore = pbpf.iloc[index + (10 + i)]['SCORE'].split(" - ")
                        
                    homediff = int(endscore[1])-int(startscore[1])
                    visitdiff = int(endscore[0])-int(startscore[0])
                    at_diff = visitdiff -homediff

                    #print(str(visitdiff -homediff))
                    #print(startscore)
                    #print(endscore)
                    btat.append([bt_diff, at_diff])
        if timeout.upper() in vistHD:

            #was there a run?
            
            if index > 10 and index + 10 < len(pbpf) :
               # print(str(index))
                for i in range(0,5):
                    if  pbpf.iloc[index -(11 + i)]['SCORE'] != None:
                        startscore = pbpf.iloc[index -(11 + i)]['SCORE'].split(" - ")
              
                if  pbpf.iloc[index -1]['SCORE'] == None:
                    break
                
                endscore = pbpf.iloc[index -1]['SCORE'].split(" - ")
                
                visitdiff = int(endscore[1])-int(startscore[1])
                homediff = int(endscore[0])-int(startscore[0])
                bt_diff = visitdiff -homediff
                
                #print(str(visitdiff -homediff))
                #print(startscore)
                #print(endscore)
                
                if index + 15 < len(pbpf) and bt_diff >= 7:

                    startscore = pbpf.iloc[index - 1]['SCORE'].split(" - ")
                    
                    for i in range(0,5):
                        if  pbpf.iloc[index + (10 + i)]['SCORE'] != None:
                            endscore = pbpf.iloc[index + (10 + i)]['SCORE'].split(" - ")
                        
                    visitdiff = int(endscore[1])-int(startscore[1])
                    homediff = int(endscore[0])-int(startscore[0])
                    at_diff = visitdiff -homediff

                    #print(str(visitdiff -homediff))
                    #print(startscore)
                    #print(endscore)
                    btat.append([bt_diff, at_diff])             
    return btat


game = game_ids[81]
pbp = playbyplayv2.PlayByPlayV2("0022200792")
pbp = pbp.get_data_frames()[0]

print("[Before timeout, After timeout]")
print(logRuns(pbp, "Hawks"))

[Before timeout, After timeout]
[]


In [24]:
#for each team pull all their cames and pull the run for each game
    #teams["id"]
    #teams["nickname"]
    
#gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2022-23',league_id_nullable='00',team_id_nullable="1610612737",season_type_nullable='Regular Season')
#print(type(gamefinder))
#games = gamefinder.get_data_frames()[0]
#game_ids = games["GAME_ID"].to_list()
#pbp = playbyplayv2.PlayByPlayV2(game)
#pbp = pbp.get_data_frames()[0]
import numpy as np
allruns = {}


for index, teams in allteams.iterrows():
    gamefinder = leaguegamefinder.LeagueGameFinder(season_nullable='2022-23',league_id_nullable='00',team_id_nullable=teams["id"],season_type_nullable='Regular Season')
    games = gamefinder.get_data_frames()[0]
    game_ids = games["GAME_ID"].to_list()
    print(teams["nickname"])
    allruns[teams["nickname"]] = []
    for game in game_ids:
        print(game)
        pbp = playbyplayv2.PlayByPlayV2(game)
        pbp = pbp.get_data_frames()[0]
        time.sleep(.3)
        runs  = logRuns(pbp, teams["nickname"])
        if len(runs) > 0:
            #print(runs)
            allruns[teams["nickname"]] .append(runs)
    print(allruns)
    ##print(allruns)
        
np.save('allruns.npy', allruns)     
    

0022200343
0022200335
0022200322
0022200306
0022200289
0022200284
0022200270
0022200255
0022200238
0022200232
0022200208
0022200195
0022200185
0022200162
0022200157
0022200143
0022200131
0022200115
0022200101
0022200087
0022200078
0022200063
0022200048
0022200040
0022200025
0022200012
{'Hawks': [[[7, -2], [8, 0]], [[10, -14]], [[22, -5]], [[9, 5], [10, 8], [8, 2]], [[10, 3], [7, -12]], [[14, -5]], [[12, 0]], [[11, -2]], [[7, -5], [16, -7]], [[10, 4], [8, -12]], [[9, 2]], [[7, 1]], [[7, -7]], [[9, -2], [9, -2]], [[14, -7]], [[10, -6]], [[9, -5]], [[11, -3]], [[9, 1]], [[8, -2]], [[7, 1]], [[8, -13]], [[7, 7], [15, 7]], [[7, 3], [7, -3]], [[7, 2], [7, -3]], [[8, 1], [12, -5]], [[10, 4], [11, -4]], [[14, -1]], [[8, -4], [8, -5]], [[9, 6]], [[7, 4], [9, 0]], [[10, 2]], [[7, 1]], [[8, -3]], [[9, -7]], [[12, 0]], [[9, -2], [8, 0], [11, 5], [13, -5]], [[34, -2]], [[15, -7]], [[7, 13], [12, 0]], [[8, -2], [7, 10]], [[9, -10]], [[12, 3], [9, -4]], [[14, -13], [7, -10]]], 'Celtics': [[[11, -5]],